In [15]:
# bring in dependencies
import pandas as pd
import numpy as np

In [162]:
year_list = ["2015", "2016", "2017", "2018", "2019", "2020", "2021"]
happiness_frame = []
happiness_df = pd.DataFrame()

file_path = "../happiness_data/"

for y in year_list:
    csv_file = file_path + y + ".csv"
#     year = y
#     print(csv_file)
    year = pd.read_csv(csv_file)
    year["yr"] = y
    
    year = year.transpose()
    year.reset_index(inplace=True)
    year.rename(columns = {"index": "col_name"}, inplace=True)
    year = year[year["col_name"].str.contains("Region|Rank|RANK|Explain|Standard|Whisker|Confidence|Dystopia|Overall|whisker") == False]
    year["new_col_name"] = np.where(year.col_name.str.contains("GDP"), "GDP per Capita",
        np.where(year.col_name.str.contains("Happiness"),"Happiness", 
        np.where(year.col_name.str.contains("Ladder"),"Happiness",
        np.where(year.col_name.str.contains("life expectancy"),"Life Expectancy",
        np.where(year.col_name.str.contains("Freedom"),"Freedom",
        np.where(year.col_name.str.contains("corruption"),"Corruption",
        np.where(year.col_name.str.contains("score"),"Happiness",
        np.where(year.col_name.str.contains("Family"),"Social Support",
        np.where(year.col_name.str.contains("yr"),"Year",
        np.where(year.col_name.str.contains("Country"),"Country",
        np.where(year.col_name.str.contains("Region"),"Region",
        np.where(year.col_name.str.contains("Corruption"),"Corruption",
        np.where(year.col_name.str.contains("Expectancy"),"Life Expectancy",
        np.where(year.col_name.str.contains("Social"),"Social Support",
        np.where(year.col_name.str.contains("Generosity"),"Generosity",
        np.where(year.col_name.str.contains("Score"),"Happiness",
        "Other"))))))))))))))))
    year.drop(columns=['col_name'], inplace=True)
    year = year.transpose()
    year.set_axis(["Country", "Happiness", "GDP per Capita", "Social Support", "Life Expectancy", "Freedom", "Corruption", "Generosity", "Year"], axis=1,inplace = True)
    year = year[:-1]
    year_df = pd.DataFrame(year)
    happiness_df = pd.concat([happiness_df, year_df])  
    
happiness_df.to_json(r'../happiness_data/happiness_json.json', orient='records')